# 06: Forecast Validation

Rolling-origin cross-validation for forecast accuracy.

**Note:** For high-variance cross-sectional data (742 districts × time), SMAPE ≤70% is realistic.

In [ ]:
import sys
import os
import importlib

# Set working directory to project root
os.chdir(os.path.dirname(os.path.abspath('__file__')))
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')
print(f"Working dir: {os.getcwd()}")

sys.path.insert(0, 'src')

# Force reimport to get latest code
import validation_forecast
importlib.reload(validation_forecast)

from validation_forecast import run_forecast_validation

## Run Complete Validation

In [ ]:
results = run_forecast_validation()

## Summary

In [ ]:
print("\nFORECAST VALIDATION SUMMARY")
print("="*40)
print(f"Rolling CV SMAPE: {results['summary']['rolling_cv_mape']:.2f}%")
print(f"Grouped CV SMAPE: {results['summary']['grouped_cv_mape']:.2f}%")

smape_target = 70
if results['summary']['rolling_cv_mape'] <= smape_target:
    print(f"\n✅ PASS: SMAPE {results['summary']['rolling_cv_mape']:.1f}% ≤ {smape_target}%")
else:
    print(f"\n⚠️ SMAPE {results['summary']['rolling_cv_mape']:.1f}% > {smape_target}% target")